# TO DO:
Edit function that creates html file to automatically delete the created file if no copes exist

In [11]:
def get_atlases ():
    from subprocess import run
    run("source ~/.bashrc", shell=True)
    output = run("atlasquery --dumpatlases", capture_output=True, universal_newlines=True, shell=True).stdout
    output = output.splitlines()
    return output


In [12]:
def find_copes (featdir):
    from subprocess import run
    output = run(f"""find {featdir} -type d -name "cope*" """, capture_output=True, universal_newlines=True, shell=True).stdout
    output = output.splitlines()
    ncopes = range(1,len(output))
    return ncopes

In [13]:
def n_contrasts (featdir, cope_n=1):
    from subprocess import run
    output = run(f"""find {featdir}/cope{cope_n}.feat -type f -name "lmax_zstat*" """, capture_output=True, universal_newlines=True, shell=True).stdout
    output = output.splitlines()
    contrasts = range(1,len(output))
    return contrasts
    


In [14]:
def load_clusters(featdir, cope, contrast):
    import pandas as pd
    clusters = pd.read_csv(f"{featdir}/cope{cope}.feat/lmax_zstat{contrast}_std.txt",sep="\t")
    return clusters

In [15]:
def label_clusters(featdir, cope=1, contrast=1, maxima=True, atlases=['Harvard-Oxford Cortical Structural Atlas','Harvard-Oxford Subcortical Structural Atlas']):
    from subprocess import run
    import pandas as pd
    outdf = pd.DataFrame(columns=['Cluster Index', 'Z', 'x', 'y', 'z', 'regions'])
    clusters = pd.read_csv(f"{featdir}/cope{cope}.feat/lmax_zstat{contrast}_std.txt",sep="\t")
    clusters_n = clusters['Cluster Index'].unique()
    html_text = ''
    for clus in clusters_n:
        regions = clusters.loc[clusters['Cluster Index'] == clus]
        if maxima:
            regions = regions.head(1)
        regions = regions.reset_index(drop=True)
        regions = regions.drop(regions.columns[5], axis = 1)
        regions['regions'] = ""
        
        for roi in range(regions.shape[0]):
            x = regions.iloc[roi]['x']
            y = regions.iloc[roi]['y']
            z = regions.iloc[roi]['z']
            output = ''
            for atlas in atlases:
                output = output + run(f"""atlasquery -a "{atlas}" -c {x},{y},{z} """, capture_output=True, universal_newlines=True, shell=True).stdout
            regions.at[roi,'regions'] = output
        outdf = outdf.append(regions)
        html_text = outdf.to_html(index = False, justify='left')
        html_text = html_text.replace('&lt;b&gt;','<b>')
        html_text = html_text.replace('&lt;/b&gt;&lt;br&gt;','</b><br>')
        html_text = html_text.replace('\\n','<br>')
        html_text = html_text.replace('\n','')
    return html_text

In [16]:
def html_report(featdir, cope=[1,], contrast=[1,], maxima=True, file = 'feat_label', outdir=False,atlases=['Harvard-Oxford Cortical Structural Atlas','Harvard-Oxford Subcortical Structural Atlas']):
    if outdir:
        filename = f"{outdir}/{file}.html"
    else:
        filename = f"{featdir}/{file}.html"
    
    html_output = open(filename, 'w')
    html_output.write("<html><head><body style=""background-color:#d2e3f4;""><title>Feat label report</title>")
    html_output.write("<h1>Feat label report</h1>")
    for i in range(len(cope)):
        html_output.write(f"<h2>Cope{cope[i]}</h2>")
        for ii in range(len(contrast)):
            html_output.write(f"<h3>Contrast{contrast[ii]}</h3>")
            html_output.write(f"""<img src="{featdir}/cope{cope[i]}.feat/rendered_thresh_zstat{contrast[ii]}.png" alt="Contrast image for cope{i}, contrast {ii}"> """)
            table = label_clusters(featdir=featdir, cope=cope[i], contrast=contrast[ii], maxima=maxima, atlases=atlases)
            html_output.write(table)
    html_output.write("</body></html>")
    html_output.close

In [18]:
featdirs = ["/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-lOFC_AC_DC_VC_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-lOFC_DS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-lOFC_VS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-mOFC_AC_DC_VC_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-mOFC_DS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-mOFC_VS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-CMPf_AC_DC_VC_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-CMPf_DS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-CMPf_VS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-MD_AC_DC_VC_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-MD_DS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-MD_VS_L.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-lOFC_AC_DC_VC_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-lOFC_DS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-lOFC_VS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-mOFC_AC_DC_VC_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-mOFC_DS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-mOFC_VS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-CMPf_AC_DC_VC_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-CMPf_DS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-CMPf_VS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-MD_AC_DC_VC_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-MD_DS_R.gfeat/",
            "/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Right-MD_VS_R.gfeat/"]
names = ["Left-lOFC_AC_DC_VC_L",
            "Left-lOFC_DS_L",
            "Left-lOFC_VS_L",
            "Left-mOFC_AC_DC_VC_L",
            "Left-mOFC_DS_L",
            "Left-mOFC_VS_L",
            "Left-CMPf_AC_DC_VC_L",
            "Left-CMPf_DS_L",
            "Left-CMPf_VS_L",
            "Left-MD_AC_DC_VC_L",
            "Left-MD_DS_L",
            "Left-MD_VS_L",
            "Right-lOFC_AC_DC_VC_R",
            "Right-lOFC_DS_R",
            "Right-lOFC_VS_R",
            "Right-mOFC_AC_DC_VC_R",
            "Right-mOFC_DS_R",
            "Right-mOFC_VS_R",
            "Right-CMPf_AC_DC_VC_R",
            "Right-CMPf_DS_R",
            "Right-CMPf_VS_R",
            "Right-MD_AC_DC_VC_R",
            "Right-MD_DS_R",
            "Right-MD_VS_R",]
mycopes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]#find_copes(featdirs[0])
for featdir, out_name in zip(featdirs, names):
    print(featdir)
    html_report(featdir, cope=mycopes, maxima=False, file = out_name)

/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-lOFC_AC_DC_VC_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-lOFC_DS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-lOFC_VS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-mOFC_AC_DC_VC_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-mOFC_DS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-mOFC_VS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-CMPf_AC_DC_VC_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-CMPf_DS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-CMPf_VS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-MD_AC_DC_VC_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-MD_DS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Group_level/Left-MD_VS_L.gfeat/
/storage/basic/neuroade/SRfMRI/preprocessed/Gr

In [ ]:
mycopes = [25,26]
html_report(featdir, cope=mycopes, maxima=False)

In [ ]:
atlases = get_atlases()
copes = find_copes(featdir)
contrasts = n_contrasts(featdir)

In [ ]:
okay = label_clusters(featdir, cope=25,maxima=False)


In [ ]:
okay

In [ ]:
html_text = html_text.replace('&lt;b&gt;','<b>')
html_text = html_text.replace('&lt;/b&gt;&lt;br&gt;','</b><br>')
html_text = html_text.replace('\\n','<br>')
html_text = html_text.replace('\n','')

In [ ]:
html_text